<a href="https://colab.research.google.com/github/pks98/NLP_Reviews/blob/main/final_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [7]:
df=pd.read_csv('final_reviews.csv')
df.head()

,Review_Title,Reviews,Rating
0,AWESOME...IN LOVE WITH IT !,Really impressed with the fastest delivery of ...,5
1,Enjoy while it lasts,This is the second time I am writing this revi...,2
2,Good,1. Quite good look2. Better noise cancellation...,4
3,not great,its a good set of ear buds but1. the quality o...,1
4,Best buds in the budget with Type C,I have been researching about getting a nice T...,5


In [8]:
df.shape

(4680, 3)

In [9]:
def data_clean(text):
  if text=='The media could not be loaded.':
    return " "
  else:
    return text

In [10]:
df['Reviews']=df['Reviews'].apply(data_clean)

In [11]:
df.fillna('',inplace=True)

In [12]:
df['text']=df['Review_Title']+' '+df['Reviews']

In [13]:
df.sample(10)

,Review_Title,Reviews,Rating,text
4586,You can buy for music but not for calling,Every thing is good but no noise cancellation ...,4,You can buy for music but not for calling Ever...
14,Below expectation,"From the first look, they really looked very n...",1,"Below expectation From the first look, they re..."
2957,Ek dam bakwas ek spiker to band hai,Mere wife ne gift Diya tha . Maine. 19March ko...,1,Ek dam bakwas ek spiker to band hai Mere wife ...
1166,Bass is only available at high volume,Sound quality is good but bass is only availab...,4,Bass is only available at high volume Sound qu...
1092,"Don't even think of this, yet another fake Chi...","1. Sound quality is just average, on scale of ...",1,"Don't even think of this, yet another fake Chi..."
3719,Mic not good.,Good sound...would have been better but consid...,2,Mic not good. Good sound...would have been bet...
1455,Very good product,Its a very good product if you looking for a g...,5,Very good product Its a very good product if y...
1548,REVIEW AFTER 2 WEEKS OF USAGE,It's been 2 weeks I bought thisPros - whatever...,4,REVIEW AFTER 2 WEEKS OF USAGE It's been 2 week...
2116,Great sound quality and poor mic performance,"Worst product, left earpiec was not working, s...",4,Great sound quality and poor mic performance W...
512,Decent but not up to the expectations,Overall the product is decent. But the noice c...,3,Decent but not up to the expectations Overall ...


In [14]:
df=df[df['Rating']!=3.0]

In [15]:
df.shape

(3987, 4)

In [16]:
def rating_classify(rating):
  if rating>=4:
    return 1
  else:
    return 0

In [17]:
df['label']=df['Rating'].apply(rating_classify)

In [18]:
df.sample(5)

,Review_Title,Reviews,Rating,text,label
4395,Somewhat ok,Bass quality 5/5Sound quality 5/5Battery life ...,4,Somewhat ok Bass quality 5/5Sound quality 5/5B...,1
643,FIRST TIME RIGHT,The first supply did not work and was replaced...,2,FIRST TIME RIGHT The first supply did not work...,0
442,Quality sound and worth purchase,Ear buds are a quality ones:Sound quality: goo...,5,Quality sound and worth purchase Ear buds are ...,1
1887,Not worth the money,Going by the reviews I purchased this airdopes...,1,Not worth the money Going by the reviews I pur...,0
4054,Not recommend at this price,"No Noise cancellation, Disconnects frequently ...",1,Not recommend at this price No Noise cancellat...,0


In [19]:
df.drop(['Review_Title','Reviews','Rating'],axis=1,inplace=True)

In [20]:
df.head()

,text,label
0,AWESOME...IN LOVE WITH IT ! Really impressed w...,1
1,Enjoy while it lasts This is the second time I...,0
2,Good 1. Quite good look2. Better noise cancell...,1
3,not great its a good set of ear buds but1. the...,0
4,Best buds in the budget with Type C I have bee...,1


In [21]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
port_stem=PorterStemmer()
def text_processing(text):
  text=re.sub('[^a-zA-Z]',' ',text)
  text=text.lower()
  text=text.split()
  # text = [port_stem.stem(word) for word in text if not word in stopwords.words('english')]
  text = [port_stem.stem(word) for word in text]
  text = ' '.join(text)
  return text

In [23]:
df['new_text']=df['text'].apply(text_processing)

In [24]:
df.head()

,text,label,new_text
0,AWESOME...IN LOVE WITH IT ! Really impressed w...,1,awesom in love with it realli impress with the...
1,Enjoy while it lasts This is the second time I...,0,enjoy while it last thi is the second time i a...
2,Good 1. Quite good look2. Better noise cancell...,1,good quit good look better nois cancel good ba...
3,not great its a good set of ear buds but1. the...,0,not great it a good set of ear bud but the qua...
4,Best buds in the budget with Type C I have bee...,1,best bud in the budget with type c i have been...


In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.layers import LSTM,Bidirectional,GRU,SpatialDropout1D
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [26]:
txt=df['new_text'].to_list()
txt

['awesom in love with it realli impress with the fastest deliveri of amazon i order it on yesterday at pm and they deliv me at pm on the same day it is awesom realli love it it doesn t fall from the ear and sound call qualiti bluetooth connect everi thing is great pleas go for it without ani singl thought',
 'enjoy while it last thi is the second time i am write thi review not sure whi my earlier detail review which wa more like a ownership review wa delet by amazon hope thi review is publish and not eventu delet howev thi review would not be as detail as the previou one purchas thi product in aug and sinc then had replac from boat within a year each unit would last to month max depend upon the usag and alway one or the other earpiec would die out the left earpiec of the rd replac die out day back pleas note i have mostli use them dure my morn jog run over all the sound qualiti is good and enjoy them as long as they last but the overal durabl of the devic is veri veri poor attach the e

In [27]:
df.shape

(3987, 3)

In [28]:
len(txt)

3987

In [29]:
y=df['label']

In [30]:
token=Tokenizer()
token.fit_on_texts(txt)

In [31]:
token.index_word

{1: 'the',
 2: 'is',
 3: 'it',
 4: 'and',
 5: 'i',
 6: 'for',
 7: 'to',
 8: 'a',
 9: 'good',
 10: 'product',
 11: 'thi',
 12: 'in',
 13: 'of',
 14: 'not',
 15: 'qualiti',
 16: 'sound',
 17: 'but',
 18: 'you',
 19: 'use',
 20: 'with',
 21: 't',
 22: 'veri',
 23: 'ear',
 24: 'on',
 25: 'call',
 26: 'are',
 27: 'one',
 28: 'have',
 29: 'my',
 30: 'boat',
 31: 'work',
 32: 'bass',
 33: 'that',
 34: 's',
 35: 'after',
 36: 'batteri',
 37: 'if',
 38: 'as',
 39: 'can',
 40: 'so',
 41: 'buy',
 42: 'charg',
 43: 'nois',
 44: 'connect',
 45: 'wa',
 46: 'at',
 47: 'time',
 48: 'also',
 49: 'bud',
 50: 'earbud',
 51: 'price',
 52: 'airdop',
 53: 'from',
 54: 'be',
 55: 'onli',
 56: 'or',
 57: 'they',
 58: 'no',
 59: 'get',
 60: 'will',
 61: 'month',
 62: 'just',
 63: 'best',
 64: 'when',
 65: 'case',
 66: 'cancel',
 67: 'day',
 68: 'other',
 69: 'go',
 70: 'replac',
 71: 'all',
 72: 'issu',
 73: 'like',
 74: 'don',
 75: 'music',
 76: 'even',
 77: 'your',
 78: 'these',
 79: 'fit',
 80: 'great',
 81

In [32]:
vocab=len(token.index_word)+1
vocab

6751

In [33]:
encoded_text=token.texts_to_sequences(txt)
encoded_text

[[110,
  12,
  163,
  20,
  3,
  81,
  463,
  20,
  1,
  2132,
  330,
  13,
  119,
  5,
  199,
  3,
  24,
  942,
  46,
  1625,
  4,
  57,
  360,
  87,
  46,
  1625,
  24,
  1,
  135,
  67,
  3,
  2,
  110,
  81,
  163,
  3,
  3,
  154,
  21,
  196,
  53,
  1,
  23,
  4,
  16,
  25,
  15,
  139,
  44,
  225,
  176,
  2,
  80,
  202,
  69,
  6,
  3,
  210,
  105,
  343,
  408],
 [522,
  83,
  3,
  151,
  11,
  2,
  1,
  285,
  47,
  5,
  96,
  315,
  11,
  93,
  14,
  335,
  409,
  29,
  813,
  765,
  93,
  92,
  45,
  99,
  73,
  8,
  3223,
  93,
  45,
  1345,
  131,
  119,
  439,
  11,
  93,
  2,
  2133,
  4,
  14,
  1753,
  1345,
  350,
  11,
  93,
  143,
  14,
  54,
  38,
  765,
  38,
  1,
  648,
  27,
  129,
  11,
  10,
  12,
  988,
  4,
  219,
  97,
  145,
  70,
  53,
  30,
  214,
  8,
  230,
  404,
  682,
  143,
  151,
  7,
  61,
  636,
  776,
  1162,
  1,
  200,
  4,
  322,
  27,
  56,
  1,
  68,
  368,
  143,
  943,
  132,
  1,
  86,
  368,
  13,
  1,
  839,
  70,
  943,
  132,


In [34]:
len(encoded_text)

3987

In [35]:
max_length=200
x=pad_sequences(encoded_text,maxlen=max_length,padding='post')

In [36]:
x

array([[110,  12, 163, ...,   0,   0,   0],
       [522,  83,   3, ...,   0,   0,   0],
       [  9, 256,   9, ...,   0,   0,   0],
       ...,
       [168,   6,  85, ...,   0,   0,   0],
       [324,  10,   1, ...,   0,   0,   0],
       [104,  31,  35, ...,   0,   0,   0]], dtype=int32)

In [37]:
x.shape

(3987, 200)

In [38]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=2,test_size=0.2,stratify=y)

In [39]:
leng=200

In [43]:
embedding_vector_features=10000
model=Sequential()
model.add(Embedding(vocab,embedding_vector_features,input_length=max_length))
# model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
# model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
# model.add(LSTM(128))
model.add(Dropout(0.5))
# model.add(GRU(64)) 
# model.add(SimpleRNN(64)) 
# model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [44]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 10000)        67510000  
                                                                 
 lstm_3 (LSTM)               (None, 200, 64)           2576640   
                                                                 
 dropout_3 (Dropout)         (None, 200, 64)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 200, 128)         66048     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 200, 128)          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                 

In [45]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=3)

Epoch 1/3
100/100 [==============================] - 603s 6s/step - loss: 0.4029 - accuracy: 0.8150 - val_loss: 0.2935 - val_accuracy: 0.9010
Epoch 2/3
100/100 [==============================] - 595s 6s/step - loss: 0.2301 - accuracy: 0.9185 - val_loss: 0.2922 - val_accuracy: 0.8922
Epoch 3/3
100/100 [==============================] - 599s 6s/step - loss: 0.1573 - accuracy: 0.9470 - val_loss: 0.3302 - val_accuracy: 0.8847
